<a href="https://colab.research.google.com/github/ekaterinatao/hackathon_books_text_classification/blob/main/inference_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Установка зависимостей

In [1]:
!pip install datasets -qqq
!pip install accelerate -U -qqq
!pip install python-docx -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
from docx import Document
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import torch
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import warnings
warnings.filterwarnings("ignore")

### Вспомогательные функции

In [3]:
def preprocess_data(examples):
    encoding = tokenizer(examples['text'], truncation=True, max_length=512)
    return encoding

def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return full_text

def create_dataframe(formatted_text):
    df = pd.DataFrame(formatted_text, columns=['text'])
    return df

### Загрузка сохраненной модели

In [7]:
# путь к предобученной модели на huggingface
checkpoint = 'iliabel/ruRoberta-large_data_new_v23'

In [9]:
# Подгрузка сохраненных весов предобученной модели
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=10, ignore_mismatched_sizes=True
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [10]:
# используемые при обучении гиперпараметры
training_args = TrainingArguments(
    output_dir='./',
    learning_rate=1e-05,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=10,
    #weight_decay=0.0001,
    evaluation_strategy="epoch",
    #push_to_hub=True,
    #report_to="wandb",
    #run_name="iliabel_run_3",
    save_strategy="no",
    group_by_length=True,
    warmup_ratio=0.1,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    #use_cpu=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    #train_dataset=encoded_dataset["train"],
    #eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

### Инференс на книге в формате `word`

In [11]:
# ПУТЬ к файлу в формате docx
input_file_path = '/content/Мужские_души_в_ПО_после_первой_читки.docx'

In [12]:
# Обрабатываем docx файл
formatted_text = read_docx(input_file_path)

# Создаем датасет из обработанных данных
df = create_dataframe(formatted_text)
dataset_book = Dataset.from_pandas(df)
dataset_book

Dataset({
    features: ['text'],
    num_rows: 1306
})

In [13]:
# кодируем датасет для загрузки в модель
encoded_book = dataset_book.map(preprocess_data, batched=True)
encoded_book.set_format("torch")
encoded_book

Map:   0%|          | 0/1306 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1306
})

In [14]:
# предсказываем метки классов
preds_book = trainer.predict(encoded_book)
preds2_book = preds_book.predictions
predicted_labels_book = np.argmax(preds2_book, axis=1)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


### Сохраняем предсказания модели и логиты

In [15]:
# метки классов исходного датасета
ind_to_label = {0: 'annotation',
                1: 'author',
                2: 'book-title',
                3: 'cite',
                4: 'epigraph',
                5: 'note',
                6: 'p',
                7: 'poem',
                8: 'subtitle',
                9: 'title',
                10: 'none'}

In [16]:
# сохраняем предсказанный класс
df['labels'] = predicted_labels_book

# сохраняем текстовые описания меток классов
df['tags'] = df['labels'].apply(lambda x: ind_to_label[x])

# сохраняем логиты (уверенность модели в предсказанном классе)
df['logits'] = np.max(preds2_book, axis=1)

In [17]:
# визуализация собранного датасета с предсказанными метками
df.sample(10)

,text,labels,tags,logits
1173,"Альберту 32 года, и он приходит на психологиче...",0,annotation,11.042936
514,,8,subtitle,7.443314
844,,8,subtitle,7.443314
855,"— Не совсем, — бормочет он.",3,cite,8.493361
1301,Шизоидное расстройство личности характеризуетс...,3,cite,11.561463
864,Следующий тезис является ключевым:,6,p,11.853862
240,ВСТАВИТЬ РИСУНОК,8,subtitle,11.266246
1177,"— Я попробую. Это нелегко, но я попытаюсь.",6,p,12.541618
51,Необходимость подробного разбора мужской дилем...,8,subtitle,11.212584
1058,В результате возникает чувство неполноценности...,0,annotation,9.793309


In [ ]:
df.to_csv('book_result.csv', index=False)

### Сохраняем результат в формате `xml`

In [18]:
# зависимости и функции
from xml.etree.ElementTree import Element, SubElement, tostring
from xml.dom import minidom

# Function to convert dataframe to one-level XML
def dataframe_to_xml(df, root_element_name="root"):
    # Create the root element
    root = Element(root_element_name)

    # Iterate over the dataframe rows and create XML elements
    for _, row in df.iterrows():
        # Create a sub-element under root with the tag from the 'tags' column
        sub_element = SubElement(root, row['tags'])
        # Set the text of this sub-element to the value from the 'text' column
        sub_element.text = str(row['text'])

    # Convert the Element tree to a string
    xml_str = minidom.parseString(tostring(root)).toprettyxml(indent="  ")
    return xml_str

# Function to save the XML to a file
def save_xml_to_file(xml_str, file_name):
    with open(file_name, 'w') as xml_file:
        xml_file.write(xml_str)

In [19]:
# путь к файлу ИЛИ подгрузка готового файла csv
file = df.copy()

In [20]:
xml = dataframe_to_xml(df)
print(xml[:1000])

<?xml version="1.0" ?>
<root>
  <title> БЬОРН ЗЮФКЕ</title>
  <subtitle/>
  <subtitle>Мужские души</subtitle>
  <subtitle>Психологический путеводитель по хрупкому миру сильного пола</subtitle>
  <cite>Все фрагменты, выделенные желтым, - это полосные врезки. они НЕ дублируются (кроме страницы 9 - это исключение). поэтому верстальщику не нужно убирать их из текста. только копировать на отдельные полосы. полосные врезки оформляем так, как в книге Digital минимализм (ITD00323852), только без боковых полос. </cite>
  <subtitle/>
  <subtitle>лучше все полосные врезки органично распределить по всей книге. чтобы не было где-то мало, а где-то полно</subtitle>
  <subtitle/>
  <note>стр. 9 - врезку после верстки из общего текста нужно удалить</note>
  <note>стр. 22 - два предложения, выделенные желтым - это ОДНА полосная врезка</note>
  <note>стр. 24 - &quot;лица, воспитывающие...&quot; и &quot;соответственно, взрослые...&quot; - это ОДНА полосная врезка</note>
  <note>стр. 46 - два предложения, 

In [ ]:
# Определяем путь для нового файла xml
file_name = 'Мужские_души_в_ПО_после_первой_читки.xml'

In [ ]:
# сохраняем XML файл
save_xml_to_file(xml, file_name)